In [10]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import time
from datetime import datetime, date
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import sys
import random
#from openpyxl import load_workbook
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
import pandas as pd
import sys
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# import Alert  
from selenium.webdriver.common.alert import Alert 
from ipywidgets import Button, Layout,FileUpload
from ipywidgets import IntProgress

In [11]:
tab = widgets.Tab()    
up = widgets.FileUpload(accept=".csv", multiple=True,counter = 0,description='選擇CSV')
# a upload box show then click it and upload the test file
up._counter = 0
#up.observe(show_content,'value')

button_upload = widgets.Button(
    description='確認上傳',
    button_style='warning',
    tooltip='Click to Upload',
    disabled=False,
    icon='check')

button_webcrawler = widgets.Button(
    description='爬地籍資料下載KML',
    button_style='info',
    disabled=True,
    tooltip='Click to webcrawler',
    icon='bug',
    layout=Layout(width='20%', height='28px'))

out = widgets.Output(layout={'border': '1px solid black'})
#
# SelectMultiple widget instantiation
eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False
)

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)



def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "application/vnd.ms-excel":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io
            
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(
             content,  
             index_col=False)
        df['sect'] = df['sect'].str.strip()
        return df
    else:
        return None

def upload():
    df = df_converter()
    up._counter = 0
    if {'town', 'sect','number','city'}.issubset(df.columns): 
        with out:
            out.clear_output()
            print('\n欄位符合，好棒棒!!\n')
            print('\n -----Your uploaded DF looks like:----- \n')
            df=df.dropna(axis=1,how="all")
            df=df.dropna(axis=0,how="all")
            #axis =1 代表欄,0代表列
            #全部都是Na的列刪掉
            button_webcrawler.description = '爬地籍資料下載KML'
            button_webcrawler.disabled = False
            if df is not None:
                print(df)
                x_axis.options = df.columns  # Dropdown Widget update
                y_axis.options = df.columns  # Dropdown Widget update
            else:
                print('Configuration is wrong/missing...')
                
    else:
        with out:
            out.clear_output()
            df=df.dropna(axis=1,how="all")
            df=df.dropna(axis=0,how="all")
            print('\n欄位有錯啦，銃三小，說明都不看清楚\n')
            button_webcrawler.description = '不給妳爬啦幹'
            button_webcrawler.disabled = True
            if df is not None:
                print(df)
                x_axis.options = df.columns  # Dropdown Widget update
                y_axis.options = df.columns  # Dropdown Widget update
            else:
                print('Configuration is wrong/missing...')
   
    return(df)
################################################################以上是介面設定###################################################

def webcrawler():
    df = upload()
    browser = webdriver.Chrome(ChromeDriverManager().install())
    info = pd.DataFrame(columns = ['city','town','sect','number','lalo'])
    for i in tqdm_notebook(range(0,len(df))):
        browser.get("https://easymap.land.moi.gov.tw/R02/Index")
        time.sleep(2)
        #browser.find_element_by_id('').click()  # 实例化Select
        try:
            browser.find_element_by_id('notShowSysinfo_id').click()
        except:
            pass
        Select(browser.find_element_by_id('select_city_id')).select_by_visible_text(df['city'][i])
        #time.sleep(4)

        browser.find_element_by_id('select_town_id').click()
        #點開下拉式選單
        Select(browser.find_element_by_id('select_town_id')).select_by_visible_text(df['town'][i])
        #點擊鄉鎮的名字
        #time.sleep(4)

        browser.find_element_by_id('select_sect_id').click()
        #點開區段下拉式選單
        s1 = Select(browser.find_element_by_id('select_sect_id'))
        #設置選取id範圍
        for each_option in s1.options:
            if df['sect'][i] in each_option.text:
                s1.select_by_visible_text(each_option.text)
        #每一個在id範圍中的值去跑，如果有跟區段名字有部分吻合的就用那個text去點擊
        browser.find_element_by_id('landno').send_keys(df['number'][i])
        #time.sleep(5)
        
        browser.find_element_by_id('mapTile_id').click()
        #點開地圖型別下拉式選單
        Select(browser.find_element_by_id('mapTile_id')).select_by_visible_text("正射影像(航照圖)")
        #選擇地圖型別
        
        browser.find_element_by_id('land_button').click()
        time.sleep(2)
        
        try:
            source = browser.find_element_by_id('OpenLayers.Layer.Vector_28_vroot')
            time.sleep(10)
            try:
                actionChains = ActionChains(browser)
                actionChains.context_click(source).perform()
                browser.find_element_by_xpath('/html/body/ul/li[2]').click()
                browser.find_element_by_id('OpenLayers.Layer.Vector_28_vroot').click()
            except:
                actionChains = ActionChains(browser)
                browser.find_element_by_id('OpenLayers.Control.PanZoomBar_43_zoomout_innerImage').click()
                actionChains.context_click(source).perform()
                browser.find_element_by_xpath('/html/body/ul/li[2]').click()
                browser.find_element_by_id('OpenLayers.Layer.Vector_28_vroot').click()

            soup7 = BeautifulSoup(browser.page_source, "html.parser") 
            blocks = soup7.find("span", {"id": "coordDisplayLonLat"})
            pngname = str(df['town'][i]) + str(df['sect'][i]) + str(df['number'][i])
            filename = "C:/Users/Lenovo/數據分析/天下專案/" + pngname + ".png"
            browser.save_screenshot(filename)
            com_info2 = pd.DataFrame({"city":[df['city'][i]], 
                            "town":[df['town'][i]],
                           "sect":[df['sect'][i]],
                           "number":df['number'][i],          
                           'lalo':[blocks.text]}) 
            info = pd.concat([info, com_info2], axis = 0)
            #用一開始建立的info來跟後面的com_info2使用 concat 合併 axis=0 為直向合併
            info = info.reset_index(drop = True)
        except:
            # 如果有弹出框 点击确定
            if alert_is_present(browser):
                alert = Alert(browser)
                alert.accept() 
         # 隨機暫停 1~5 秒
        time.sleep(random.uniform(1, 5))
        
        
        
    info[['latitude','longitude']] = info.lalo.str.split(",",expand=True,)
    info['latitude'] = pd.to_numeric(info['latitude'])
    info['longitude'] = pd.to_numeric(info['longitude'])
    info.to_excel("area_df_clean.xlsx")
    #Open the file to be written.
    f = open('測試.kml','w',encoding = 'UTF-8')

    #Writing the kml file.
    f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
    f.write("<kml xmlns='http://earth.google.com/kml/2.1'>\n")
    f.write("<Document>\n")
    for index, row in info.iterrows():
        f.write("   <Placemark>\n")
        f.write("       <name>" + str(row['city'])+str(row['town'])+str(row['sect'])+str(row['number']) + "</name>\n")
        f.write("       <description>" + str(row['city'])+str(row['town'])+str(row['sect'])+str(row['number'])+ "</description>\n")
        f.write("       <Point>\n")
        f.write("           <coordinates>" + str(row['longitude']) + "," + str(row['latitude']) + ","  + "</coordinates>\n")
        #因為,是文字，所以不能直接用數字加上去，要轉換成string
        f.write("       </Point>\n")
        f.write("   </Placemark>\n")
    f.write("</Document>\n")
    f.write("</kml>\n")
    f.close()
    

def alert_is_present(browser):
    try:
        alert = browser.switch_to.alert
        alert.text
        return alert
    except:
        return False

        
            
def preview_clicked(b):
    preview()
def webcrawler_clicked(b):
    webcrawler()
def upload_clicked(b):
    upload()



In [12]:
# Assigning functionality to buttons
button_upload.on_click(upload_clicked)
button_webcrawler.on_click(webcrawler_clicked)

accordion = widgets.Accordion(children=[
    up])

accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_upload, button_webcrawler]),
    out
])
tab.set_title(0, "Upload")


In [13]:
accordion_box

ValueError: Could not get version for Chrome with this command: google-chrome --version || google-chrome-stable --version